## Introduction

I started as a Data Scientist at a startup, A42 Labs, in November and hit the ground running FAST in the world of data science consulting. I was quickly onboarded and rolled into a client project as the lead Data Scientist within a week of starting. The experience has been highly challenging and rewarding, with exponential improvements in my Python and SQL code. Perhaps the most benefitial aspect of this first client project has been the need to optimize code to fit within the parameters of our local programming requirements. This post highlights the approaches used to eliminate code loops, and prepare client data for row-wise regex string comparisons. 

## The Problem

The client in question (Client A), is a large, Fortune-ranked software company. There are plugins that can be utilized with their software that are developed either internally or by 3rd party partners. These plugins represent a large revenue stream for Client A, engage thousands of partners who develop and sell these plugins, hundreds of thousands of customers who purchase and use them, and cover a wide range of functionality and industries. Internal teams at Client A would like to leverage this data to report utilization and revenue back to partners, and build machine learning models to make plugin recommendations to customers. 

However, through many iterations of legacy data systems, historical data quality issues have occurred. Specifically, links between the plugin data and the partners who own/create the plugins have been lost, and no unique key exists between the data. Working with client-side SMEs, A42 Labs built a rule-based algorithm to match nearly 13,000 plugins with nearly 12,000 potential partners using regex string matching between the plugin information and the partner information where available. Due to security constraints from Client A, this process had to be conducted on a local enviroment, specifically a MacBook Pro with 16GB of RAM. This led to initial code iterations that could take more than a day to run. Since this was a highly iterative process working with SMEs to validate and update the algorithm matching rules, improving code efficiency was a priority when I started on the project to eliminate bottlenecks. 

## The Solution

The rule-based algorithm was built to match plugins to their partners based on information found in the plugin and partner registration data. This repeatable, semi-automated process for matching plugins to partners relied on automatically identifying a set of candidate partner matches using regular expression string searching. Since Python Pandas is designed for vectorized manipulations it's important to avoid the use of loops in the code logic. Pandas reads all data into memory, so looping over rows is very inefficient. The A42 Labs team decided to leverage the apply method to compare plugin and partner string information in a row-wise, vectorized fashion. An additional benefit of using this method is that, once the solution is sent to the client side production team, it can more easily be converted to a parallel process, such as using PySpark, since row-wise operations can be sharded across parallel processes. To execute this logic, we built a pairwise comparison table of all plugins matched with all possible partners. 

This process alone was eating A LOT of computation time by time I joined the project. Here's a breakdown of how optimized the creation of the pairwise comparison table. 

### Imports

Since this project used proprietary data from Client A, we'll illustrate the process using some fake data. [Faker](https://faker.readthedocs.io/en/master/) is an awesome Python library for creating fake profile data! I found out about Faker from Khuyen Tran, an amazing data science blogger and programmer. Check out all her great work [here](https://khuyentran1401.github.io/). 

In [29]:
import numpy as np
import pandas as pd
import time
from faker import Faker
fake = Faker()

### Create fake data

In [55]:
profiles = [fake.profile() for i in range(1000)]
profiles = pd.DataFrame(profiles)

### Split data

We'll create 1000 fake profiles and then "break" the data into two dataframes to simulate the disjointed data from our project with Client A. We will also take 95% of the records from dataframe 2 to simulate the unequal sizes of our original data from Client A. 

In [70]:
#Split dataframe into 2 dataframes
data1 = profiles.iloc[:,[0,1,2,3,4,5,6,7]]
data2 = profiles.iloc[:,[8,9,10,11,12]]
data2 = data2.sample(frac = 0.95)

In [71]:
data1.head()

,job,company,ssn,residence,current_location,blood_group,website,username
0,Biomedical engineer,Johnson-Nielsen,498-02-9147,"89354 Lane Port Apt. 601\nEast Reginachester, ...","(88.788543, 138.653825)",O+,"[https://pace.com/, https://www.green-golden.c...",danielbush
1,Psychiatric nurse,Erickson-Sandoval,042-58-6461,Unit 5985 Box 1994\nDPO AP 55641,"(-62.932539, 75.463822)",O-,"[http://phillips.biz/, http://rich-foster.com/...",deborah56
2,Retail banker,"Ashley, Smith and Smith",304-94-3140,"2675 Sara Wells\nEast Marychester, AR 38705","(88.7125995, 160.992887)",B+,"[https://www.hunt.info/, http://www.friedman.o...",durangeorge
3,Cartographer,Turner-Mann,265-76-4909,724 Christopher Crest Suite 315\nEast Samantha...,"(-26.503493, 18.161378)",AB-,"[https://www.berry.info/, https://www.wright-f...",ljones
4,Nature conservation officer,"Rangel, Mahoney and Cohen",218-30-1766,"3787 Smith Points\nMooretown, KS 58975","(-71.339396, 166.394354)",O-,"[http://www.mendoza-nelson.com/, http://www.ri...",kellysmith


In [72]:
data2.head()

,name,sex,address,mail,birthdate
520,Bryan Frazier,M,"6496 Veronica Lane\nDawnberg, VT 93759",ashley88@hotmail.com,1935-11-13
941,Jeffrey Mcdonald,M,"671 Shelby Mission\nNew Hannahland, AL 89586",bradymark@gmail.com,1926-07-09
137,Lisa Clark,F,"45540 Debra Views Apt. 453\nLake Daniel, CO 65869",ohardy@yahoo.com,1999-09-13
410,Becky Curtis,F,"51620 Ashley Park Suite 123\nSouth Gary, AK 89507",rachel00@gmail.com,2014-01-29
689,Walter Ortiz,M,"1487 Barber Rapids Suite 411\nLake Melissa, MO...",ericsawyer@hotmail.com,2019-07-01


Now that we've created our fake data sources, let's see how large our pairwise comparison table will be:

In [73]:
# Number of plugins
print(data1.shape)

# Number of partners
print(data2.shape)

print(data1.shape[0] * data2.shape[0])

(1000, 8)
(950, 5)
950000


We'll be creating a pairwise comparison table of every row from dataset 1 matched with every row of dataset 2. This is 950,000 records. For context, our problem with Client A required comparing nearly 13,000 rows of plugins with nearly 12,000 potential partners - that's a pairwise comparison table of nearly 155 million!

### Method 1: Append

When I first started at A42 Labs in November and started taking over the code base for this project, the existing code relied on nested for-loops, itterows, and append to create the pairwise comparison table, and output the table to a csv in chunks. The itterows and append methods, and the repeated IO operations lead to succesful yet computational inefficent process. Given the size of the data for Client A, this took over 18 hours to run; a process that was not sustainable for the required iterations. On our small size of fake data below this method takes over 5 minutes. 

In [74]:
%%time
cols = ['job',
 'company',
 'ssn',
 'residence',
 'current_location',
 'blood_group',
 'website',
 'username',
 'name',
 'sex',
 'address',
 'mail',
 'birthdate']

pairwise_lst = []
rowcnt = 0

# append rows in batches of 15000
for i_out, row_out in data1.iterrows():
    for i_in, row_in in data2.iterrows():
        pairwise_lst.append(row_out.append(row_in))
        rowcnt += 1
        if rowcnt % 15000 == 0:
            pairwise = pd.DataFrame(pairwise_lst,columns = cols)
            pairwise.to_csv('pairwise_comparison.csv', mode='a', header=False, index=False)
            pairwise_lst = []
            
pairwise = pd.read_csv('pairwise_comparison.csv', header=None, names = cols)
print(pairwise.shape)
pairwise.head()

(4170000, 13)
CPU times: user 5min 58s, sys: 1.84 s, total: 6min
Wall time: 6min


,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,"Merchandiser, retail","Martinez, Murray and Kennedy",465-91-8267,"86808 Angela Knoll Suite 262\nGoodmanmouth, KS...","(Decimal('41.901851'), Decimal('-145.313007'))",B+,"['https://www.phillips.com/', 'https://may.com...",heidi38,Robert Maddox,M,"409 Smith Drive Suite 355\nLake Brandi, AZ 10037",julia02@hotmail.com,1973-02-08
1,"Merchandiser, retail","Martinez, Murray and Kennedy",465-91-8267,"86808 Angela Knoll Suite 262\nGoodmanmouth, KS...","(Decimal('41.901851'), Decimal('-145.313007'))",B+,"['https://www.phillips.com/', 'https://may.com...",heidi38,Meghan Calderon,F,"97979 Todd Lodge Suite 012\nMichaelview, NM 88452",connersteven@gmail.com,1994-01-07
2,"Merchandiser, retail","Martinez, Murray and Kennedy",465-91-8267,"86808 Angela Knoll Suite 262\nGoodmanmouth, KS...","(Decimal('41.901851'), Decimal('-145.313007'))",B+,"['https://www.phillips.com/', 'https://may.com...",heidi38,Emma Hawkins,F,"PSC 0275, Box 0301\nAPO AP 59374",mariejones@hotmail.com,1913-06-18
3,"Merchandiser, retail","Martinez, Murray and Kennedy",465-91-8267,"86808 Angela Knoll Suite 262\nGoodmanmouth, KS...","(Decimal('41.901851'), Decimal('-145.313007'))",B+,"['https://www.phillips.com/', 'https://may.com...",heidi38,Nathaniel Johnson,M,"PSC 3310, Box 3880\nAPO AE 42018",bradleykennedy@hotmail.com,1919-05-10
4,"Merchandiser, retail","Martinez, Murray and Kennedy",465-91-8267,"86808 Angela Knoll Suite 262\nGoodmanmouth, KS...","(Decimal('41.901851'), Decimal('-145.313007'))",B+,"['https://www.phillips.com/', 'https://may.com...",heidi38,Jason Wong,M,"613 Townsend Port\nPaulchester, SD 06065",daniellebishop@hotmail.com,1948-05-07


### Method 2: Merge

The more I understood the project and data requirements, the more I realized that the proposed pairwise comparison table was simply a cartesian join of the two disjoint datasets. Assigning both datasets an arbitrary key value of 1, we can merge using an outer join on that key. 

In [75]:
%%time
pairwise_merge = data1.assign(key=1).merge(data2.assign(key=1), how='outer', on='key')
print(pairwise_merge.shape)
pairwise_merge.head()

(950000, 14)
CPU times: user 102 ms, sys: 14.8 ms, total: 116 ms
Wall time: 116 ms


,job,company,ssn,residence,current_location,blood_group,website,username,key,name,sex,address,mail,birthdate
0,Biomedical engineer,Johnson-Nielsen,498-02-9147,"89354 Lane Port Apt. 601\nEast Reginachester, ...","(88.788543, 138.653825)",O+,"[https://pace.com/, https://www.green-golden.c...",danielbush,1,Bryan Frazier,M,"6496 Veronica Lane\nDawnberg, VT 93759",ashley88@hotmail.com,1935-11-13
1,Biomedical engineer,Johnson-Nielsen,498-02-9147,"89354 Lane Port Apt. 601\nEast Reginachester, ...","(88.788543, 138.653825)",O+,"[https://pace.com/, https://www.green-golden.c...",danielbush,1,Jeffrey Mcdonald,M,"671 Shelby Mission\nNew Hannahland, AL 89586",bradymark@gmail.com,1926-07-09
2,Biomedical engineer,Johnson-Nielsen,498-02-9147,"89354 Lane Port Apt. 601\nEast Reginachester, ...","(88.788543, 138.653825)",O+,"[https://pace.com/, https://www.green-golden.c...",danielbush,1,Lisa Clark,F,"45540 Debra Views Apt. 453\nLake Daniel, CO 65869",ohardy@yahoo.com,1999-09-13
3,Biomedical engineer,Johnson-Nielsen,498-02-9147,"89354 Lane Port Apt. 601\nEast Reginachester, ...","(88.788543, 138.653825)",O+,"[https://pace.com/, https://www.green-golden.c...",danielbush,1,Becky Curtis,F,"51620 Ashley Park Suite 123\nSouth Gary, AK 89507",rachel00@gmail.com,2014-01-29
4,Biomedical engineer,Johnson-Nielsen,498-02-9147,"89354 Lane Port Apt. 601\nEast Reginachester, ...","(88.788543, 138.653825)",O+,"[https://pace.com/, https://www.green-golden.c...",danielbush,1,Walter Ortiz,M,"1487 Barber Rapids Suite 411\nLake Melissa, MO...",ericsawyer@hotmail.com,2019-07-01


Our operation has now improved from nearly 6 minutes to 145 milliseconds, a 2500% improvement! 

### Method 3: Concatenation

Another method I found thanks to every programmers favorite resource, StackOverflow, was to concatenate every row from one dataset onto the other dataset. This replicates rows of the first dataframe by the length of the second dataframe, and then concatenates the data along their columns. 

In [76]:
%%time
def pairwise_merge_concat(*args):
    if len(args) == 2:
        if len(args[0]) > len(args[1]):
            df_1 = pd.concat([args[0]] * len(args[1]), ignore_index=True).sort_values(by=[args[0].columns[0]]).reset_index(drop=True)
            df_2 = pd.concat([args[1]] * len(args[0]), ignore_index=True)
            return pd.concat([df_1, df_2], 1)
        
pairwise_concat = pairwise_merge_concat(data1, data2)
print(pairwise_concat.shape)
pairwise_concat.head()

(950000, 13)
CPU times: user 879 ms, sys: 47 ms, total: 926 ms
Wall time: 919 ms


,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Academic librarian,Hernandez Group,529-11-4629,"296 Cantu Well Apt. 012\nSalazarville, MO 13321","(-62.432317, -159.991382)",A-,[http://www.carrillo.com/],ngilbert,Bryan Frazier,M,"6496 Veronica Lane\nDawnberg, VT 93759",ashley88@hotmail.com,1935-11-13
1,Academic librarian,Archer-Johnston,684-80-9437,"PSC 5995, Box 2224\nAPO AP 44095","(-61.533478, -15.740287)",AB+,[https://white.com/],hector63,Jeffrey Mcdonald,M,"671 Shelby Mission\nNew Hannahland, AL 89586",bradymark@gmail.com,1926-07-09
2,Academic librarian,Williams-Rose,552-95-8900,"66392 Logan Coves\nPort Vickieberg, VA 63363","(-24.9732075, -156.640190)",A+,"[http://www.duncan-chavez.biz/, https://www.ri...",lgriffin,Lisa Clark,F,"45540 Debra Views Apt. 453\nLake Daniel, CO 65869",ohardy@yahoo.com,1999-09-13
3,Academic librarian,Williams-Rose,552-95-8900,"66392 Logan Coves\nPort Vickieberg, VA 63363","(-24.9732075, -156.640190)",A+,"[http://www.duncan-chavez.biz/, https://www.ri...",lgriffin,Becky Curtis,F,"51620 Ashley Park Suite 123\nSouth Gary, AK 89507",rachel00@gmail.com,2014-01-29
4,Academic librarian,Archer-Johnston,684-80-9437,"PSC 5995, Box 2224\nAPO AP 44095","(-61.533478, -15.740287)",AB+,[https://white.com/],hector63,Walter Ortiz,M,"1487 Barber Rapids Suite 411\nLake Melissa, MO...",ericsawyer@hotmail.com,2019-07-01


Concatenation gives us similar computation cost savings as the merge method. I ended up using this concatenation because I felt it could be more flexible with different size data frames in the future. While the original method took over 18 hours in the Client A project, this improved code only took 7.5 minutes! I was certainly excited to get my workday back and not have to worry about running this section of code overnight. 

## Conclusion

Even in early stages of a project, thinking critically about data structures and how data will need to be used further down the data processing and algorithm development pipeline can lead to vast improvements in computational time requirements and mitigation of pipeline bottlenecks. 